In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime


tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!rm -rf ./logs/GRU/ #should not run this line it will remove the tensorboard logs

In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename_a, filename_b):
    with open(filename_b, 'rb') as f:
        labels = np.load(f).ravel()
    
    data = pd.read_pickle(filename_a)
    #https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
    data_x = data.pivot(index='id', columns='time', values='FLUX')
    data_x = data_x.to_numpy()
    data_x = convert_3D(normalize(data_x))
    return data_x, labels

In [5]:
training_data_x, training_data_y = features_and_labels("../Kepler-Train.pkl", "../Labels-Train.npy")
testing_data_x, testing_data_y = features_and_labels("../Kepler-Test.pkl", "../Labels-Test.npy")
validation_data_x, validation_data_y = features_and_labels("../Kepler-Val.pkl", "../Labels-Val.npy")

In [8]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU')

Epoch 1/10
394/394 [==============================] - 15s 22ms/step - loss: 0.6802 - accuracy: 0.5822 - auc_1: 0.5021 - precision: 0.1080 - recall: 0.1133 - false_negatives: 2559.0000 - false_positives: 2701.0000 - true_negatives: 7002.0000 - true_positives: 327.0000 - val_loss: 0.6640 - val_accuracy: 0.7370 - val_auc_1: 0.5078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 361.0000 - val_false_positives: 53.0000 - val_true_negatives: 1160.0000 - val_true_positives: 0.0000e+00
Epoch 2/10
394/394 [==============================] - 7s 16ms/step - loss: 0.6427 - accuracy: 0.7696 - auc_1: 0.5693 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_negatives: 2886.0000 - false_positives: 14.0000 - true_negatives: 9689.0000 - true_positives: 0.0000e+00 - val_loss: 0.6167 - val_accuracy: 0.7706 - val_auc_1: 0.6745 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 361.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 1213.0000 -

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [9]:
%tensorboard --logdir logs/GRU/fit